In [3]:
# !pip install geojson
# !pip install folium
# !pip install geopandas
# !pip install pyogrio

In [4]:
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import json
import folium
import geopandas as gpd
import pyogrio
from shapely.ops import split
from shapely.geometry import MultiLineString, LineString, Point, Polygon, MultiPolygon
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster

import requests
import json


### DB IGN : Routes, Points d'intérêts, Habitations

### Loading and understanding

In [5]:
file_ign_44 = '../Data/BDCARTO/44_Loire_Atlantique/1_DONNEES_LIVRAISON_2024-12-00080/data.gpkg'
df_layers = gpd.list_layers(file_ign_44)

c:\Users\eliot\anaconda3\Lib\site-packages\pyogrio\core.py:130: RuntimeWarning: Field format 'character varying' not supported
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))
c:\Users\eliot\anaconda3\Lib\site-packages\pyogrio\core.py:130: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))
c:\Users\eliot\anaconda3\Lib\site-packages\pyogrio\core.py:130: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))
c:\Users\eliot\anaconda3\Lib\site-packages\pyogrio\core.py:130: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_list_layers(get_vsi_path_or_buffer(path_or_buffer))


### Contour de la Baule

In [7]:
# Charger un fichier GeoPackage
gdf_commune = gpd.read_file(file_ign_44, layer="commune")

In [8]:
# Le code postal a l'air d'être un identifiant unique
gdf_la_baule = gdf_commune[gdf_commune['code_postal'] == '44500']
gdf_la_baule

,cleabs,code_insee,code_insee_de_l_arrondissement,code_insee_de_la_collectivite_terr,code_insee_du_departement,code_insee_de_la_region,population,surface_en_ha,code_postal,nom_officiel,chef_lieu_d_arrondissement,chef_lieu_de_collectivite_terr,chef_lieu_de_departement,chef_lieu_de_region,capitale_d_etat,date_du_recensement,organisme_recenseur,lien_vers_chef_lieu,code_siren,geometry
266,COMMUNE_0000000009745827,44055,3,44D,44,52,16468,2219,44500,La Baule-Escoublac,False,False,False,False,False,2021-01-01,INSEE,PAIHABIT0000000028858387,214400558,"MULTIPOLYGON (((300057.1 6702628.1, 300040.5 6..."


In [10]:
### Converting all geopds to GPS coordinate
gdf_la_baule_gps = gdf_la_baule.to_crs("EPSG:4326")
ign_geometry_la_baule_gps = gdf_la_baule_gps.iloc[0].geometry

In [11]:

# Centrer la carte sur La Baule (approx.)
center_lat, center_lon = ign_geometry_la_baule_gps.centroid.y, ign_geometry_la_baule_gps.centroid.x
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Ajouter le contour de la ville
folium.GeoJson(
    ign_geometry_la_baule_gps,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)


# Ajouter un contrôle des couches
folium.LayerControl().add_to(m)

# Afficher la carte
m

## Données carroyées

In [12]:
# 1min30 à loader environ
file_carroye = '../Data/Filosofi2017_carreaux_200m_gpkg/Filosofi2017_carreaux_200m_met.gpkg'
df_layers = gpd.list_layers(file_carroye)
df_layers
gdf_carroye = gpd.read_file(file_carroye)
print(len(gdf_carroye))
gdf_carroye.head()

2294776


,Idcar_200m,I_est_200,Idcar_1km,I_est_1km,Idcar_nat,Groupe,Ind,Men_1ind,Men_5ind,Men_prop,...,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc,Men_pauv,Men,lcog_geo,geometry
0,CRS3035RES200mN2029800E4254200,1,CRS3035RES1000mN2029000E4254000,1,CRS3035RES8000mN2024000E4248000,14863,2.0,0.3,0.0,0.6,...,0.0,0.4,0.3,0.7,0.1,0.0,0.2,0.9,2A041,"POLYGON ((1220156.068 6049788.964, 1220138.321..."
1,CRS3035RES200mN2030000E4254200,1,CRS3035RES1000mN2030000E4254000,1,CRS3035RES8000mN2024000E4248000,14863,1.0,0.2,0.0,0.3,...,0.0,0.2,0.1,0.4,0.1,0.0,0.1,0.4,2A041,"POLYGON ((1220138.321 6049989.597, 1220120.574..."
2,CRS3035RES200mN2030000E4254600,1,CRS3035RES1000mN2030000E4254000,1,CRS3035RES8000mN2024000E4248000,14863,5.0,0.9,0.1,1.5,...,0.1,1.1,0.7,1.8,0.3,0.0,0.4,2.2,2A041,"POLYGON ((1220536.283 6050025.179, 1220518.536..."
3,CRS3035RES200mN2030200E4254200,1,CRS3035RES1000mN2030000E4254000,1,CRS3035RES8000mN2024000E4248000,14863,3.0,0.5,0.1,0.9,...,0.1,0.6,0.4,1.1,0.2,0.0,0.3,1.3,2A041,"POLYGON ((1220120.574 6050190.229, 1220102.827..."
4,CRS3035RES200mN2030400E4253800,1,CRS3035RES1000mN2030000E4253000,1,CRS3035RES8000mN2024000E4248000,175938,2.0,0.2,0.1,0.7,...,0.2,0.6,0.5,0.4,0.1,0.0,0.1,0.9,2A041,"POLYGON ((1219704.866 6050355.277, 1219687.119..."


* Dictionnaire des variables dispo sur : https://www.insee.fr/fr/statistiques/6215138?sommaire=6215217#dictionnaire
* Données qui nous intéressent : 
    * Ind (nb individus), Men (nb de ménages), Men_mais (nb de ménages en maison) 
    * I_est_200 (imputé)

In [13]:
print(gdf_carroye.crs)
gdf_carroye_gps = gdf_carroye.copy()
gdf_carroye_gps = gdf_carroye_gps.to_crs(epsg=4326)
gdf_carroye_city_gps = gdf_carroye_gps[gdf_carroye_gps.geometry.intersects(ign_geometry_la_baule_gps)]
len(gdf_carroye_city_gps)

EPSG:2154


431

### Analyse rapide

#### Données imputées quand le nb de foyers fiscaux est peut être inférieur à 11

In [63]:
# Zones mesurées
gdf_carroye_city_gps[gdf_carroye_city_gps.I_est_200 == 0].Ind.describe()

count    271.000000
mean      64.459410
std       37.006559
min       16.000000
25%       40.000000
50%       56.500000
75%       74.250000
max      284.500000
Name: Ind, dtype: float64

In [14]:
# Zones estimées
# gdf_carroye_city_gps[gdf_carroye_city_gps.I_est_200 == 1].Ind.describe()
gdf_carroye_city_gps[gdf_carroye_city_gps.I_est_200 == 1].Men.describe()

count    160.000000
mean       4.024375
std        3.650566
min        0.400000
25%        1.200000
50%        2.650000
75%        6.225000
max       16.500000
Name: Men, dtype: float64

In [65]:
# Niveau de vie moyen par personne (très grosse approximation ici)
gdf_carroye_city_gps['Ind_snv_per_ind'] = gdf_carroye_city_gps['Ind_snv']  / gdf_carroye_city_gps['Ind'] 
# gdf_carroye_city_gps.Ind_snv.describe()
gdf_carroye_city_gps.Ind_snv_per_ind.describe()

c:\Users\eliot\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


count      431.000000
mean     28022.033898
std       4086.999613
min      15877.006678
25%      25011.433333
50%      28142.937500
75%      30642.449926
max      38105.407692
Name: Ind_snv_per_ind, dtype: float64

### Données manquantes grisées

In [16]:
import branca

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the city boundary contour
folium.GeoJson(
    ign_geometry_la_baule_gps,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Create a colormap based on the number of people in each carreau
min_people = gdf_carroye_city_gps['Ind'].min()  # Minimum number of people
max_people = gdf_carroye_city_gps['Ind'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_people, max_people)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in gdf_carroye_city_gps.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    num_people = row['Ind']  # Assuming this column exists
    num_menages = row['Men']  # Assuming this column exists
    cond = row['I_est_200'] == 1

    # Create a popup displaying the number of people
    text = f"Number of people: {num_people} \n Number of ménages: {num_menages}"
    popup = folium.Popup(text, max_width=300)
    
    # print(num_people)
    

    # Style each feature based on the number of people
    if not cond:
        folium.GeoJson(
            feature,
            name=f"Carreau {row.name}",
            popup=popup,
            style_function=lambda x, num_people=num_people: {
                    'fillColor': colormap(num_people),  # Color based on the number of people
                    'color': 'black',  # Border color
                    'weight': 2,  # Border width
                    'fillOpacity': 0.7  # Fill opacity
                }
        ).add_to(m)
    
    else:
        folium.GeoJson(
            feature,
            name=f"Carreau {row.name}",
            popup=popup,
            style_function=lambda x: {
                'fillColor': 'black',  # You can change this or use a color scale
                'color': 'black',  # You can change this or use a color scale
                'weight': 2,
                'fillOpacity': 0.7
            }
        ).add_to(m)

# Return the map to visualize
m

### Toutes les données

In [ ]:
import branca

m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the city boundary contour
folium.GeoJson(
    ign_geometry_la_baule_gps,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Create a colormap based on the number of people in each carreau
min_people = gdf_carroye_city_gps['Ind'].min()  # Minimum number of people
max_people = gdf_carroye_city_gps['Ind'].max()  # Maximum number of people
colormap = branca.colormap.linear.YlOrRd_09.scale(min_people, max_people)  # Yellow to red color scale

# Add the colormap legend to the map
colormap.add_to(m)

# Iterate through each row in the GeoDataFrame and plot each feature
for _, row in gdf_carroye_city_gps.iterrows():
    # Extract geometry and properties for each feature
    feature = row['geometry']
    num_people = row['Ind']  # Assuming this column exists
    num_menages = row['Men']  # Assuming this column exists
    cond = row['I_est_200'] == 1

    # Create a popup displaying the number of people
    text = f"Number of people: {num_people} \n Number of ménages: {num_menages}"
    popup = folium.Popup(text, max_width=300)
    
    # print(num_people)
    

    # Style each feature based on the number of people
    folium.GeoJson(
        feature,
        name=f"Carreau {row.name}",
        popup=popup,
        style_function=lambda x, num_people=num_people: {
                'fillColor': colormap(num_people),  # Color based on the number of people
                'color': 'black',  # Border color
                'weight': 2,  # Border width
                'fillOpacity': 0.7  # Fill opacity
            }
    ).add_to(m)

# Return the map to visualize
m